<a href="https://colab.research.google.com/github/Kh0uloud/Fine-Grained-Sentiment-Analysis-for-Gym-Customer-Feedback/blob/main/Clustering_Aspects.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install umap-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 5.1 MB/s eta 0:00:00


In [2]:
import pandas as pd
import torch
import umap
import time
import numpy as np
import plotly.graph_objects as go

from sklearn.cluster import KMeans

from collections import Counter
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from sklearn.preprocessing import LabelEncoder
import re
import itertools

In [29]:
df = pd.read_csv("/content/drive/MyDrive/GYMProject/all_aspect_sentiment_counts.csv")

In [30]:
df = df[["term", "Total", "adequate %", "basic %", "conflict %", "impressive %", "limitation %", "negative %", "neutral %", "none %", "positive %"]]

In [31]:
df

,term,Total,adequate %,basic %,conflict %,impressive %,limitation %,negative %,neutral %,none %,positive %
0,staff,7520,0.0,0.0,0.04,0.0,0.0,20.35,0.40,0.00,79.22
1,equipment,7155,0.0,0.0,0.15,0.0,0.0,28.86,1.33,0.00,69.66
2,noaspectterm,3628,0.0,0.0,0.00,0.0,0.0,2.87,0.33,96.11,0.69
3,classes,2383,0.0,0.0,0.04,0.0,0.0,19.22,2.43,0.00,78.30
4,machines,1791,0.0,0.0,0.06,0.0,0.0,51.42,2.85,0.00,45.67
...,...,...,...,...,...,...,...,...,...,...,...
9336,24 hour access through pods,1,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.00,100.00
9337,yanga,1,0.0,0.0,0.00,0.0,0.0,100.00,0.00,0.00,0.00
9338,yanga drinks station,1,0.0,0.0,0.00,0.0,0.0,100.00,0.00,0.00,0.00
9339,yanga sport water,1,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.00,100.00


In [ ]:
!git clone https://github.com/openai/CLIP.git

Cloning into 'CLIP'...
remote: Enumerating objects: 256, done.
remote: Total 256 (delta 0), reused 0 (delta 0), pack-reused 256 (from 1)
Receiving objects: 100% (256/256), 8.93 MiB | 18.69 MiB/s, done.
Resolving deltas: 100% (133/133), done.


In [ ]:
!pip install ftfy regex tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.2 MB/s eta 0:00:00


In [ ]:
%cd /content/CLIP

/content/CLIP


In [ ]:
import clip

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/16", device=device)

text = clip.tokenize(df['term'].tolist()).to(device)

with torch.no_grad():
    text_features = model.encode_text(text)

100%|███████████████████████████████████████| 335M/335M [00:08<00:00, 39.6MiB/s]


In [ ]:
# Add the terms embeddings into the dataframe
df['text_features'] = text_features.cpu().tolist()

In [ ]:
start = time.time()
projections = umap.UMAP(n_components=3,init='random').fit_transform(text_features.cpu())
end = time.time()

print(f"generating projections with UMAP took: {(end-start):.2f} sec")
print(projections.shape)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


generating projections with UMAP took: 48.52 sec
(9341, 3)


In [ ]:
# Extract X, Y, Z coordinates
x, y, z = projections[:, 0], projections[:, 1], projections[:, 2]

# Create the 3D scatter plot
fig = go.Figure()

terms = df['term'].tolist()  # Simulated text terms
terms = np.array(terms)

fig.add_trace(go.Scatter3d(
    x=projections[:, 0],
    y=projections[:, 1],
    z=projections[:, 2],
    mode='markers',
    marker=dict(
        size=6,              # Point size
        color=projections[:, 2],  # Color based on Z-axis
        colorscale='viridis',  # Color map
        opacity=0.8
    ),
    text=terms,  # Display term on hover
    hoverinfo="text",  # Only show text on hover
))

# Update layout
fig.update_layout(
    title="3D Projection of Embeddings",
    scene=dict(
        xaxis_title="X-axis",
        yaxis_title="Y-axis",
        zaxis_title="Z-axis"
    ),
    margin=dict(l=0, r=0, b=0, t=40)
)

# Show the interactive figure
fig.show()

In [ ]:
kmeans = KMeans(n_clusters=50, random_state=42)
kmeans.fit(df['text_features'].tolist())
centroids = kmeans.cluster_centers_

clusters = kmeans.predict(df['text_features'].tolist())

In [ ]:
df[df['cluster'] == 0]['term'].tolist()

Texts in Cluster 0:
- mats
- bill
- harry
- fraser
- mangers
- dean
- josh
- matts
- luke
- chris
- jack
- james
- aaron
- mox
- chantelle
- dave
- eddie
- sue
- carys
- libby
- liam
- nathan
- toliets
- ollie
- odie
- joshs
- kelly
- kassidy chaplin
- jenna
- phil
- rachael
- barry
- shaun
- bays
- bell
- david
- dreia
- dumbles
- ellies
- emily
- deannas
- eoin
- deepas
- deputy
- de riguer
- ewen
- ethans
- erce
- dave shadlock
- darcen
- darren
- darren briggs
- female pt
- curtis
- fraser pt
- craig
- craige
- gareth
- garth
- gabs
- george
- gosia
- godfray
- godfrey
- guy david
- handkes
- hatti
- hayden
- hayley
- hannah
- harriet
- harvey
- helen
- help canter
- clientel
- inductor jen
- john
- jermaine
- jess
- jess swabey
- jimy rae
- james induction
- jake gadsden
- james brereton
- joseph
- kelly smith
- kassidy
- kellie
- kellie jones
- kellie pt
- karis
- karl
- kieron
- kit ide
- kyle
- kyrun
- laurence
- lee steweat
- leahs
- leanne
- leigham
- lento
- lizzie
- lily
- 

In [ ]:
df[df['cluster'] == 1]['term'].tolist()

Texts in Cluster 1:
- changing rooms
- toilets
- changing room
- toilet
- changing facilities
- bathrooms
- bathroom
- ladies toilets
- mens changing rooms
- ladies changing room
- mens changing room
- male changing room
- toilet facilities
- change rooms
- mens toilets
- disabled toilet
- changing room facilities
- ladies changing rooms
- male changing rooms
- toilet area
- ladies room
- female changing room
- female toilets
- male toilets
- mens locker room
- mens toilet
- gents toilets
- womens toilets
- ladies toilet
- washrooms
- changing rooms/showers
- mens bathroom
- female changing rooms
- womens changing room
- women’s changing rooms
- locker room facilities
- gents changing room
- toilets/changing rooms
- toilet door
- toilet areas
- mens locker rooms
- restrooms
- toilet cleanliness
- changing facility
- changing/shower room
- men changing room
- dressing room
- disabled bathroom
- dressing rooms
- disabled toilets
- male toilet
- changing cubicles
- changing facilities for

In [ ]:
# Function to preprocess and get the top N most common n-grams (bigrams, trigrams, etc.)
def preprocess_and_get_top_n_terms(cluster_texts, n=4, n_gram=2):
    # Preprocess the texts (lowercase and remove stopwords)
    processed_texts = []
    for text in cluster_texts:
        # Remove non-alphabetical characters and split by spaces
        words = re.findall(r'\b\w+\b', text.lower())
        # Remove stopwords
        filtered_words = [word for word in words if word not in ENGLISH_STOP_WORDS]
        processed_texts.append(filtered_words)

    # Create n-grams from the words
    n_grams = []
    for words in processed_texts:
        n_grams.extend([' '.join(gram) for gram in zip(*[words[i:] for i in range(n_gram)])])

    # Count frequency of n-grams
    n_gram_counts = Counter(n_grams)

    # Get the top N most common n-grams
    most_common_terms = n_gram_counts.most_common(n)

    return most_common_terms

cls = 0
cluster_texts = df[df['cluster'] == cls]['term'].tolist()
top_terms = preprocess_and_get_top_n_terms(cluster_texts, n=5, n_gram=1)
top_terms.extend(preprocess_and_get_top_n_terms(cluster_texts, n=5, n_gram=2))

# Print the top 4 most common n-grams (bigrams) and their occurrences
for term, count in top_terms:
    print(f"'{term}': {count} occurrences")

'phone': 29 occurrences
'wifi': 26 occurrences
'customer': 20 occurrences
'service': 18 occurrences
'mobile': 12 occurrences
'wi fi': 10 occurrences
'customer service': 10 occurrences
'customer support': 3 occurrences
'phone signal': 2 occurrences
'wifi connection': 2 occurrences


In [15]:
cluster_topic = ['nonvalid',
                 'changing room',
                 'machines',
                 'covid',
                 'equipment',
                 'service',
                 'classes',
                 'hygiene products',
                 'rooms/space',
                 'enviroment',
                 'deadlift',
                 'gym system: access',
                 'price and payment policy',
                 'staff',
                 'showers',
                 'training and personal training',
                 'bike',
                 'cable machines',
                 'nonvalid',
                 'classes',
                 'cleaning',
                 'machines',
                 'drinks and water machines',
                 'benches',
                 'gym (variety)',
                 'membership',
                 'machines',
                 'parking',
                 'weights',
                 'dumbbells and boxing',
                 'nonvalid',
                 'nonvalid',
                 'leg equipment',
                 'trainers',
                 'cardio',
                 'lockers',
                 'floor',
                 'music',
                 'nonvalid',
                 'wifi',
                 'opening hours',
                 'class',
                 'ladies area',
                 'water in showers and pools',
                 'machine',
                 'shower',
                 'space',
                 'paper',
                 'air conditioner',
                 'entrance']

len(cluster_topic)

50

In [ ]:
df.to_csv("/content/drive/MyDrive/GYMProject/aspect_sentiment_.csv", index=True)

In [13]:
df

,Unnamed: 0,term,Total,adequate %,basic %,conflict %,impressive %,limitation %,negative %,neutral %,none %,positive %,text_features,cluster
0,0,staff,7520,0.0,0.0,0.04,0.0,0.0,20.35,0.40,0.00,79.22,"[0.32763671875, -0.194091796875, -0.2177734375...",13
1,1,equipment,7155,0.0,0.0,0.15,0.0,0.0,28.86,1.33,0.00,69.66,"[0.08770751953125, -0.106689453125, -0.2491455...",4
2,2,noaspectterm,3628,0.0,0.0,0.00,0.0,0.0,2.87,0.33,96.11,0.69,"[0.0028400421142578125, -0.35107421875, -0.175...",30
3,3,classes,2383,0.0,0.0,0.04,0.0,0.0,19.22,2.43,0.00,78.30,"[0.134033203125, 0.44091796875, -0.2158203125,...",19
4,4,machines,1791,0.0,0.0,0.06,0.0,0.0,51.42,2.85,0.00,45.67,"[-0.358154296875, -0.28759765625, -0.357910156...",2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9336,9336,24 hour access through pods,1,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.00,100.00,"[-0.19091796875, -0.2481689453125, -0.15478515...",40
9337,9337,yanga,1,0.0,0.0,0.00,0.0,0.0,100.00,0.00,0.00,0.00,"[0.11737060546875, -0.60205078125, -0.20251464...",30
9338,9338,yanga drinks station,1,0.0,0.0,0.00,0.0,0.0,100.00,0.00,0.00,0.00,"[0.01021575927734375, -0.036468505859375, -0.0...",22
9339,9339,yanga sport water,1,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.00,100.00,"[-0.098388671875, -0.140625, -0.1461181640625,...",22


In [25]:
import pandas as pd
df= pd.read_csv("/content/drive/MyDrive/GYMProject/aspect_sentiment_.csv")

list = ['impressive', 'negative', 'positive', 'basic', 'conflict', 'none', 'neutral', 'adequate', 'limitation']
df1 = pd.read_csv("/content/drive/MyDrive/GYMProject/all_aspect_sentiment_counts.csv")
for l in list:
    df[l] = df1[l]

# Group by 'cluster' and sum the sentiment counts
cluster_groups = df.groupby('cluster')[['Total', 'adequate', 'basic', 'conflict', 'impressive', 'limitation',
                                        'negative', 'neutral', 'none', 'positive']].sum()

# Compute the percentage of each sentiment per cluster based on the total reviews in that cluster
for col in ['adequate', 'basic', 'conflict', 'impressive', 'limitation', 'negative', 'neutral', 'none', 'positive']:
    cluster_groups[f'{col} %'] = (cluster_groups[col] / cluster_groups['Total']) * 100

# Assign cluster topics (ensure the length matches the number of clusters)
cluster_groups['cluster_topic'] = cluster_topic

# Reorganize the DataFrame to include 'cluster_topic' first
final_df = cluster_groups[['cluster_topic', 'Total', 'adequate %', 'basic %', 'conflict %',
                           'impressive %', 'limitation %', 'negative %', 'neutral %', 'none %', 'positive %']]

# Display the resulting dataframe
print(final_df)

                          cluster_topic  Total  adequate %  basic %  \
cluster                                                               
0                              nonvalid    342    0.000000   0.0000   
1                         changing room   1971    0.050736   0.0000   
2                              machines   2304    0.000000   0.0000   
3                                 covid    272    0.000000   0.0000   
4                             equipment   8766    0.000000   0.0000   
5                               service   2045    0.000000   0.0489   
6                               classes    751    0.000000   0.0000   
7                      hygiene products    545    0.000000   0.0000   
8                           rooms/space    521    0.000000   0.0000   
9                            enviroment   6443    0.000000   0.0000   
10                             deadlift    357    0.000000   0.0000   
11                   gym system: access    497    0.000000   0.0000   
12    

In [26]:
# Aggregate sentiments into Positive, Negative, and Neutral
cluster_groups['Positive %'] = cluster_groups[['adequate %', 'impressive %', 'positive %']].sum(axis=1)
cluster_groups['Negative %'] = cluster_groups[['conflict %', 'limitation %', 'negative %']].sum(axis=1)
cluster_groups['Neutral %'] = cluster_groups[['basic %', 'neutral %', 'none %']].sum(axis=1)

In [34]:
result = cluster_groups[['cluster_topic', 'Total', 'Positive %', 'Negative %', 'Neutral %']]
result = result[result['cluster_topic'] != 'nonvalid'].sort_values(by='Total', ascending=False)

In [38]:
result

,cluster_topic,Total,Positive %,Negative %,Neutral %
cluster,,,,,
13,staff,9705,70.077280,29.242658,0.680062
4,equipment,8766,71.708875,27.024869,1.266256
9,enviroment,6443,69.005122,28.713332,2.281546
19,classes,2935,76.763203,21.090290,2.146508
24,gym (variety),2705,72.495379,24.731978,2.772643
42,ladies area,2529,74.970344,23.013049,2.016607
2,machines,2304,46.484375,50.303819,3.211806
5,service,2045,56.625917,40.831296,2.542787
1,changing room,1971,24.657534,72.805682,2.536783


In [40]:
result[result['Positive %'] > 70]

,cluster_topic,Total,Positive %,Negative %,Neutral %
cluster,,,,,
13,staff,9705,70.077280,29.242658,0.680062
4,equipment,8766,71.708875,27.024869,1.266256
19,classes,2935,76.763203,21.090290,2.146508
24,gym (variety),2705,72.495379,24.731978,2.772643
42,ladies area,2529,74.970344,23.013049,2.016607
33,trainers,1842,74.647123,24.267101,1.085776
15,training and personal training,823,76.792224,18.347509,4.860267


In [42]:
result[result['Negative %'] > 50]

,cluster_topic,Total,Positive %,Negative %,Neutral %
cluster,,,,,
2,machines,2304,46.484375,50.303819,3.211806
1,changing room,1971,24.657534,72.805682,2.536783
14,showers,1640,19.695122,77.926829,2.378049
28,weights,1422,31.997187,60.759494,7.243319
25,membership,1275,32.392157,58.666667,8.941176
48,air conditioner,1079,11.862836,86.468953,1.668211
26,machines,812,20.689655,70.443350,8.866995
37,music,764,26.963351,70.026178,3.010471
43,water in showers and pools,697,11.047346,83.213773,5.738881
